# CSE 251B Project Milestone Problem 2 Code

In [2]:
# from starter file
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch

import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid 
import os
import time
import tqdm
import itertools


train_npz = np.load('./train.npz')
train_data = train_npz['data']
test_npz  = np.load('./test_input.npz')
test_data  = test_npz['data']

X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

class TrajectoryDatasetTrain(Dataset):
    def __init__(self, data, scale=10.0, augment=True):
        """
        data: Shape (N, 50, 110, 6) Training data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        augment: Whether to apply data augmentation (only for training)
        """
        self.data = data
        self.scale = scale
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx]
        # Getting 50 historical timestamps and 60 future timestamps
        hist = scene[:, :50, :].copy()    # (agents=50, time_seq=50, 6)
        future = torch.tensor(scene[0, 50:, :2].copy(), dtype=torch.float32)  # (60, 2)
        
        # Data augmentation(only for training)
        if self.augment:
            if np.random.rand() < 0.5:
                theta = np.random.uniform(-np.pi, np.pi)
                R = np.array([[np.cos(theta), -np.sin(theta)],
                              [np.sin(theta),  np.cos(theta)]], dtype=np.float32)
                # Rotate the historical trajectory and future trajectory
                hist[..., :2] = hist[..., :2] @ R
                hist[..., 2:4] = hist[..., 2:4] @ R
                future = future @ R
            if np.random.rand() < 0.5:
                hist[..., 0] *= -1
                hist[..., 2] *= -1
                future[:, 0] *= -1

        # Use the last timeframe of the historical trajectory as the origin
        origin = hist[0, 49, :2].copy()  # (2,)
        hist[..., :2] = hist[..., :2] - origin
        future = future - origin

        # Normalize the historical trajectory and future trajectory
        hist[..., :4] = hist[..., :4] / self.scale
        future = future / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            y=future.type(torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )

        return data_item
    

class TrajectoryDatasetTest(Dataset):
    def __init__(self, data, scale=10.0):
        """
        data: Shape (N, 50, 110, 6) Testing data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        """
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Testing data only contains historical trajectory
        scene = self.data[idx]  # (50, 50, 6)
        hist = scene.copy()
        
        origin = hist[0, 49, :2].copy()
        hist[..., :2] = hist[..., :2] - origin
        hist[..., :4] = hist[..., :4] / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )
        return data_item

torch.manual_seed(251)
np.random.seed(42)

scale = 7.0

N = len(train_data)
val_size = int(0.1 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


### Problem 2A - Searching for the optimal parameters to train our model on

In [ ]:
def hyperparameter_search(
    train_data: np.ndarray,
    model_class,
    model_kwargs: Dict = {},
    search_params: Dict = None,
    batch_size: int = 32,
    val_ratio: float = 0.1,
    scale: float = 7.0,
    max_epochs: int = 30,
    early_stopping_patience: int = 5,
    device: torch.device = None,
    results_dir: str = 'hyperparameter_search_results',
    verbose: bool = True
):
    """
    Perform hyperparameter search for trajectory prediction model.
    
    Args:
        train_data: Training data
        model_class: Model class to instantiate
        model_kwargs: Additional model parameters
        search_params: Dictionary of parameters to search over
        batch_size: Batch size for training
        val_ratio: Proportion of data to use for validation
        scale: Scale factor for normalization
        max_epochs: Maximum number of epochs to train
        early_stopping_patience: Number of epochs to wait before early stopping
        device: Device to train on
        results_dir: Directory to save results
        verbose: Whether to print progress
        
    Returns:
        results_df: DataFrame of results
        best_params: Best parameters found
    """
    if device is None:
        if torch.backends.mps.is_available():
            device = torch.device('mps')
            print("Using Apple Silicon GPU")
        elif torch.cuda.is_available():
            device = torch.device('cuda')
            print("Using CUDA GPU")
        else:
            device = torch.device('cpu')
            print("Using CPU")
    
    if search_params is None:
        search_params = {
            'learning_rate': [1e-2, 1e-3, 5e-4],
            'weight_decay': [1e-4, 1e-5, 0.0],
            'step_size': [10, 20, 30],
            'gamma': [0.5, 0.25, 0.1]
        }
    
    # Create directory for results
    os.makedirs(results_dir, exist_ok=True)
    
    # Prepare the data
    N = len(train_data)
    val_size = int(val_ratio * N)
    train_size = N - val_size
    
    # Create datasets
    train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
    val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)
    
    # Create data loaders
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=lambda x: Batch.from_data_list(x)
    )
    
    val_dataloader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        collate_fn=lambda x: Batch.from_data_list(x)
    )
    
    # Generate parameter combinations
    param_grid = list(ParameterGrid(search_params))
    if verbose:
        print(f"Testing {len(param_grid)} parameter combinations")
    
    # Initialize results tracking
    results = []
    
    # Run hyperparameter search
    for param_idx, params in enumerate(param_grid):
        if verbose:
            print(f"\nParameter set {param_idx+1}/{len(param_grid)}: {params}")
        
        # Create model
        model = model_class(**model_kwargs).to(device)
        
        # Setup optimizer and scheduler
        optimizer = optim.Adam(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
        
        scheduler = optim.lr_scheduler.StepLR(
            optimizer,
            step_size=params['step_size'],
            gamma=params['gamma']
        )
        
        # Track best validation loss
        best_val_loss = float('inf')
        no_improvement = 0
        criterion = nn.MSELoss()
        
        # Track metrics
        train_losses = []
        val_losses = []
        val_maes = []
        val_mses = []
        epoch_times = []
        
        # Training loop
        for epoch in range(max_epochs):
            start_time = time.time()
            
            # ---- Training ----
            model.train()
            train_loss = 0
            for batch in train_dataloader:
                batch = batch.to(device)
                pred = model(batch)
                y = batch.y.view(batch.num_graphs, 60, 2)
                loss = criterion(pred, y)
                
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                optimizer.step()
                
                train_loss += loss.item()
            
            # ---- Validation ----
            model.eval()
            val_loss = 0
            val_mae = 0
            val_mse = 0
            with torch.no_grad():
                for batch in val_dataloader:
                    batch = batch.to(device)
                    pred = model(batch)
                    y = batch.y.view(batch.num_graphs, 60, 2)
                    val_loss += criterion(pred, y).item()
                    
                    # Unnormalize for metric calculation
                    pred_unnorm = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
                    y_unnorm = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
                    
                    val_mae += nn.L1Loss()(pred_unnorm, y_unnorm).item()
                    val_mse += nn.MSELoss()(pred_unnorm, y_unnorm).item()
            
            # Calculate average losses
            train_loss /= len(train_dataloader)
            val_loss /= len(val_dataloader)
            val_mae /= len(val_dataloader)
            val_mse /= len(val_dataloader)
            
            # Step scheduler
            scheduler.step()
            
            # Track time
            epoch_time = time.time() - start_time
            epoch_times.append(epoch_time)
            
            # Track metrics
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            val_maes.append(val_mae)
            val_mses.append(val_mse)
            
            if verbose:
                print(f"Epoch {epoch:03d} | LR {optimizer.param_groups[0]['lr']:.6f} | "
                      f"Train MSE {train_loss:.4f} | Val MSE {val_loss:.4f} | "
                      f"Val MAE {val_mae:.4f} | Val ADE {val_mae:.4f} | "
                      f"Time {epoch_time:.2f}s")
            
            # Check for improvement
            if val_loss < best_val_loss - 1e-3:
                best_val_loss = val_loss
                no_improvement = 0
            else:
                no_improvement += 1
                if no_improvement >= early_stopping_patience:
                    if verbose:
                        print(f"Early stopping at epoch {epoch}")
                    break
        
        # Record results for this parameter set
        avg_epoch_time = sum(epoch_times) / len(epoch_times)
        best_epoch = val_losses.index(min(val_losses))
        
        param_results = {
            **params,
            'best_val_loss': min(val_losses),
            'best_val_mae': val_maes[best_epoch],
            'best_val_mse': val_mses[best_epoch],
            'best_epoch': best_epoch,
            'epoch_time_seconds': avg_epoch_time,
            'total_epochs': len(train_losses)
        }
        
        results.append(param_results)
        
        # Create and save training curve plot
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 2, 1)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title('Training Curves')
        plt.legend()
        
        plt.subplot(2, 2, 2)
        plt.plot(val_maes, label='MAE (meters)')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.title('Validation MAE')
        
        plt.subplot(2, 2, 3)
        lr_values = [optimizer.param_groups[0]['lr'] * (params['gamma'] ** (i // params['step_size'])) 
                     for i in range(len(train_losses))]
        plt.plot(lr_values)
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.title('Learning Rate Schedule')
        
        plt.subplot(2, 2, 4)
        plt.bar(['Learning Rate', 'Weight Decay', 'Step Size', 'Gamma'], 
                [params['learning_rate'], params['weight_decay'], params['step_size'], params['gamma']])
        plt.title('Hyperparameters')
        plt.yscale('log')
        
        plt.tight_layout()
        plt.savefig(f"{results_dir}/params_{param_idx+1}_plot.png")
        plt.close()
        
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Sort by validation MAE
    results_df = results_df.sort_values('best_val_mae')
    
    # Save results
    results_df.to_csv(f"{results_dir}/hyperparameter_search_results.csv", index=False)
    
    # Get best parameters
    best_params = results_df.iloc[0].to_dict()
    
    # Plot comparative results
    plt.figure(figsize=(15, 10))
    
    param_labels = [f"lr={p['learning_rate']:.0e}, wd={p['weight_decay']:.0e}, ss={p['step_size']}, γ={p['gamma']}" 
                   for _, p in results_df.iloc[:10].iterrows()]
    
    plt.subplot(2, 2, 1)
    plt.barh(param_labels, results_df['best_val_mae'].iloc[:10])
    plt.xlabel('MAE (meters)')
    plt.title('Top 10 Parameter Sets by MAE')
    
    plt.subplot(2, 2, 2)
    plt.barh(param_labels, results_df['best_val_mse'].iloc[:10])
    plt.xlabel('MSE (square meters)')
    plt.title('MSE for Top 10 Parameter Sets')
    
    plt.subplot(2, 2, 3)
    plt.barh(param_labels, results_df['epoch_time_seconds'].iloc[:10])
    plt.xlabel('Average Epoch Time (seconds)')
    plt.title('Training Time for Top 10 Parameter Sets')
    
    plt.subplot(2, 2, 4)
    plt.barh(param_labels, results_df['best_epoch'].iloc[:10])
    plt.xlabel('Best Epoch')
    plt.title('Convergence Speed for Top 10 Parameter Sets')
    
    plt.tight_layout()
    plt.savefig(f"{results_dir}/comparative_results.png")
    plt.close()
    
    if verbose:
        print("\nHyperparameter search complete!")
        print(f"Best parameters: {best_params}")
    
    return results_df, best_params


### Problem 2B - Deciding which model we should use to make predictions

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim=50 * 50 * 2, output_dim=60 * 2):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, data):
        x = data.x[..., :2] # (batch*50, 50, 2)
        x = x.reshape(-1, 50 * 50 * 2) # (batch, 5000)
        x = self.linear(x)
        return x.view(-1, 60, 2)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_features, output_features):
        super(MLP, self).__init__()
        
        # Define the layers
        self.flatten = nn.Flatten()
        self.mlp = nn.Sequential(
            nn.Linear(input_features, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, output_features)
        )
    
    def forward(self, data):
        x = data.x
        if len(x.shape) == 4:
            x = x[:, :, :, :2] # (batch, 50, 50, 2)
            # x = x.reshape(-1, 50 * 50 * 6)
            x = x.reshape(x.size(0), -1) # (batch, 50 * 50 * 6)
        else:
            x = self.flatten(x)
        x = self.mlp(x)
        return x.view(-1, 60, 2)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=128, output_dim=60 * 2):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x
        x= x.reshape(-1, 50, 50, 6)  # (batch_size, num_agents, seq_len, input_dim)
        x = x[:, 0, :, :] # Only Consider ego agent index 0

        lstm_out, _ = self.lstm(x)
        # lstm_out is of shape (batch_size, seq_len, hidden_dim) and we want the last time step output
        out = self.fc(lstm_out[:, -1, :])
        return out.view(-1, 60, 2)